In [17]:
#패키지 로드 
import pandas as pd
import glob
import os
import numpy as np
import re
import time
from tqdm import tqdm_notebook as tqdm
import sys
from konlpy.tag import Okt
from gensim.models import Word2Vec
from gensim.models import KeyedVectors
import matplotlib.pyplot as plt
from ckonlpy.tag import Twitter
from ckonlpy.tag import Postprocessor
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
import sklearn.metrics as metrics
import sklearn as skl
from sklearn.metrics import classification_report,confusion_matrix
from sklearn.neighbors import KNeighborsClassifier
from collections import Counter

In [2]:
#통합 및 정리본 불러오기
data = pd.read_excel('C:/Users/USER/Desktop/labelingdata2018.xlsx', index_col=0)
print(len(data))
data = data.dropna(axis = 0) 
print(len(data)) #결측치 하나 있음

FileNotFoundError: [Errno 2] No such file or directory: 'C:/Users/USER/Desktop/labelingdata2018.xlsx'

In [55]:
twi = Twitter()
add_noun = ['비대면','확진자','거리두기','사랑제일교회','의료기관','의료진','집단감염','유연근무','유연근무제','유연근로제','한국판뉴딜','디지털뉴딜',
            '그린뉴딜','휴먼뉴딜','취약계층','긴급재난지원금','고용보험','진료거부','택배기사','선별진료소','고용보험','특수고용직','특고','재배포',
            '고용안정지원금','공유오피스','공용오피스','저소득층','실업급여','유통업계','물류센터','소상공인','유연근로','특수형태근로자','특수형태근로종사자']
#새로운 단어 등록 하기
for i in add_noun:
    twi.add_dictionary(i,'Noun')
    
#통합어 만들기
replace = {'유연근로제':'유연근무제','유연근무':'유연근무제','유연근로':'유연근무제','특고':'특수고용직','특수형태근로자':'특수고용직','특수형태근로종사자':'특수고용직'}
postprocessor = Postprocessor(base_tagger=twi, replace=replace, passtags={'Noun'})

#형태소 분석 하기
words = [[j[i][0] for i in range(len(j))] for j in [postprocessor.pos(i) for i in tqdm(data['text'])]]



C:\Users\USER\anaconda3\lib\site-packages\konlpy\tag\_okt.py:16: UserWarning: "Twitter" has changed to "Okt" since KoNLPy v0.4.5.
  warn('"Twitter" has changed to "Okt" since KoNLPy v0.4.5.')
C:\Users\USER\anaconda3\lib\site-packages\ipykernel_launcher.py:14: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  


In [56]:
#배제어 등록하기
Prohibit_words = ['기자','연합뉴스','뉴시스','시사저널','신문','뉴스','사진','헤럴드경제','노컷뉴스','파이낸셜뉴스','특파원',
                  '라며','대해','지난','위해','오전','오후','무단','배포','이데일리','머니투데이','앵커','지금','때문','이번',
                  '통해','정도','경우','관련','이미지','출처','일보','바로가기','까지','여개','도록','이나','재배포','처럼','면서',
                  '거나','이제','지난달','어요']
#배제어 제거, 한 글자 제거하기
j = 0
for i in tqdm(words):
    for k in Prohibit_words:
        while k in i:
            i.remove(k)
    words[j] = i
    j += 1 #불용어 제외

for k in range(len(words)):
    words[k] = [i for i in words[k] if len(i) > 1]  # 한글자 제외
    
data['words'] = words


C:\Users\USER\anaconda3\lib\site-packages\ipykernel_launcher.py:8: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  


In [14]:
#저장 및 불러오기
#data.to_excel('C:/Users/USER/Desktop/labelingdata2018_word.xlsx')
data =  pd.read_excel('C:/Users/USER/Desktop/labelingdata2018_word.xlsx', index_col=0)#데이터 불러오기
data['words'] = [i.replace("'", "").replace('[', '').replace(']', '').replace(' ', '').split(',') for i in
          list(data['words'])]  # words가 텍스트 형식으로 되어 있을 경우
data = data.drop_duplicates(['text'])
data = data.reset_index()
'''
검증은 크게 두 가지로 한다.
1. 기존 모델을 이용하여 검증
    (기존 모델은 3~8월 일별 무작위 추출하여 고용과 전혀 연관이 없는 데이터도 굉장히 많음)
2. 현재 데이터로 w2v 모델을 만들어서 검증 
    (현재 데이터 = labelling data 2018 이며, 포함어를 통해 1차 정제한 기사로, 모든 기사들이 고용관련 키워드 1단어 이상씩 포함되어 있음)

'''
#테스트 
data_test = data[0:1000]
print(len(data))
print(len(data.drop_duplicates(['text'])))

words = data['words']
type(words[0])

23930
23930


list

In [58]:
# 1 번 모델 검증

#기존 모델 불러오기
embedding_model = KeyedVectors.load_word2vec_format('C:/Users/USER/Desktop/1~8newsurl/w2v') # 모델 로드

word_vector = embedding_model.wv.vectors
match_index = embedding_model.wv.index2word
vecdata = pd.DataFrame(columns=['x{}'.format(i) for i in range(100)])
for i in tqdm(range(len(data_test['words']))):
    vec = pd.DataFrame(columns = ['x{}'.format(i) for i in range(100)])
    j = 0
    for w in match_index:
        if w in words[i]:
            vec.loc[j] = list(word_vector[match_index.index(w)])
            j += 1
        score_vec = vec.mean()
    vecdata.loc[i] = score_vec



C:\Users\USER\anaconda3\lib\site-packages\ipykernel_launcher.py:6: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  
C:\Users\USER\anaconda3\lib\site-packages\ipykernel_launcher.py:7: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  import sys
C:\Users\USER\anaconda3\lib\site-packages\ipykernel_launcher.py:9: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  if __name__ == '__main__':


In [59]:
data_all_1 = pd.concat([data_test,vecdata],axis = 1)
data_all_1.describe()


,user1,user2,user3,user4,user5,user6,sum,y,x0,x1,...,x90,x91,x92,x93,x94,x95,x96,x97,x98,x99
count,1000.000000,1000.000000,1000.000000,1000.00000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,...,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000
mean,0.165000,0.252000,0.172000,0.11800,0.388000,0.404000,1.499000,0.178000,0.205235,-0.335677,...,-0.558566,0.436400,0.017514,0.079786,-0.215654,-0.273091,-0.497571,-0.270289,0.134462,-0.179111
std,0.371366,0.434379,0.377569,0.32277,0.487538,0.490943,1.794218,0.382704,0.456961,0.474661,...,0.605068,0.717967,0.447868,0.565562,0.559525,0.582632,0.639628,0.603774,0.698603,0.585644
min,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,-1.801135,-1.955624,...,-2.431135,-1.454807,-2.070603,-1.786882,-2.184879,-3.242843,-2.821828,-2.524566,-2.268699,-2.008883
25%,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,-0.105568,-0.662403,...,-0.991585,-0.099085,-0.239289,-0.328291,-0.606281,-0.653194,-0.992088,-0.560309,-0.301675,-0.583602
50%,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,1.000000,0.000000,0.172821,-0.300934,...,-0.613155,0.399951,0.040072,0.104507,-0.168695,-0.258192,-0.490504,-0.230253,0.174552,-0.128930
75%,0.000000,1.000000,0.000000,0.00000,1.000000,1.000000,3.000000,0.000000,0.480012,-0.034365,...,-0.186970,0.896735,0.282418,0.475533,0.177004,0.080237,-0.035904,0.164661,0.649284,0.239867
max,1.000000,1.000000,1.000000,1.00000,1.000000,1.000000,6.000000,1.000000,2.114494,1.381521,...,1.992224,2.929128,1.999513,2.480282,1.418137,1.988677,1.487228,1.259800,1.648804,1.926397


In [62]:
#  기존 모델 학습
data_all = pd.read_csv('C:/Users/USER/Desktop/data_all.csv')

x_train = data_all.iloc[:,6:106]
y_train = data_all.iloc[:,107]

x_test = data_all_1.iloc[:,12:112]
y_test = data_all_1.iloc[:,10]


#x_train 정규화
scaler = skl.preprocessing.StandardScaler()
scaler.fit(x_train) #x_train 데이터에 대해서 평균과 표준편차 계산
x_train = scaler.transform(x_train)
x_test = scaler.transform(x_test)
###########knn사용해서 분류해보기

classifier = KNeighborsClassifier(n_neighbors= 2)
classifier.fit(x_train,y_train)
knn_pre = classifier.predict(x_test)

#성능 평가
print(confusion_matrix(y_test,knn_pre))

#평가결과 확인
print(classification_report(y_test,knn_pre)) #'''
print(classifier.score(x_test, y_test))

#뉴스 데이터 정확도 0

[[822   0]
 [178   0]]
              precision    recall  f1-score   support

           0       0.82      1.00      0.90       822
           1       0.00      0.00      0.00       178

    accuracy                           0.82      1000
   macro avg       0.41      0.50      0.45      1000
weighted avg       0.68      0.82      0.74      1000



C:\Users\USER\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


0.822


In [15]:
words = data['words']
# 2 번 모델 검증
embedding_model = Word2Vec(words, size=100, window = 6, min_count=100, workers=4, iter=100)

word_vector = embedding_model.wv.vectors
match_index = embedding_model.wv.index2word
vecdata = pd.DataFrame(columns=['x{}'.format(i) for i in range(100)])
for i in tqdm(range(len(data['words']))):
    vec = pd.DataFrame(columns = ['x{}'.format(i) for i in range(100)])
    j = 0
    for w in match_index:
        if w in words[i]:
            vec.loc[j] = list(word_vector[match_index.index(w)])
            j += 1
        score_vec = vec.mean()
    vecdata.loc[i] = score_vec

data_all_2 = pd.concat([data,vecdata],axis = 1)
data_all_2.to_excel('C:/Users/USER/Desktop/labelingdata2018_w2v.xlsx')

C:\Users\USER\anaconda3\lib\site-packages\ipykernel_launcher.py:8: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  


In [25]:
data_all_2 = data_all_2.dropna(axis = 0) 

In [33]:
#2번모델 knn돌려보기
x = data_all_2.iloc[:,13:113]
y = data_all_2.iloc[:,11]

x_train, x_test, y_train, y_test = skl.model_selection.train_test_split(x, y,random_state=100,stratify=y) #stratify = y y를 동일한 비율로 나누어 준다.
#정규화시키는 함수
scaler = skl.preprocessing.StandardScaler()

#x_train 정규화
scaler.fit(x_train) #x_train 데이터에 대해서 평균과 표준편차 계산
x_train = scaler.transform(x_train)
x_test = scaler.transform(x_test)
###########knn사용해서 분류해보기

classifier = KNeighborsClassifier(n_neighbors= 3)
classifier.fit(x_train,y_train)
knn_pre = classifier.predict(x_test)

#성능 평가
print(confusion_matrix(y_test,knn_pre))

#평가결과 확인
print(classification_report(y_test,knn_pre)) #'''
print(classifier.score(x_test, y_test))


[[5161  215]
 [ 320  287]]
              precision    recall  f1-score   support

           0       0.94      0.96      0.95      5376
           1       0.57      0.47      0.52       607

    accuracy                           0.91      5983
   macro avg       0.76      0.72      0.73      5983
weighted avg       0.90      0.91      0.91      5983

0.9105799766003677


In [90]:
# 2 번 모델 검증
embedding_model = Word2Vec(words, size=100, window = 6, min_count=100, workers=4, iter=100)

word_vector = embedding_model.wv.vectors
match_index = embedding_model.wv.index2word
vecdata = pd.DataFrame(columns=['x{}'.format(i) for i in range(100)])
for i in tqdm(range(len(data['words']))):
    vec = pd.DataFrame(columns = ['x{}'.format(i) for i in range(100)])
    j = 0
    for w in match_index:
        if w in words[i]:
            vec.loc[j] = list(word_vector[match_index.index(w)])
            j += 1
        score_vec = vec.mean()
    vecdata.loc[i] = score_vec

data_all_2 = pd.concat([data,vecdata],axis = 1)


326

In [135]:
import operator
word_list = ["123", "테스트", "test", "123", "테스트", "test123", "중복", "중복"]
result = Counter(words[0])
print(result)
#print(result.keys())
#print(result.values())
aa = dict(result)
sdict = sorted(aa.items(),key=operator.itemgetter(1), reverse = True)
sdict[1][0]

for i in range(int(len(sdict) * 0.3)):
    print(sdict[i][0])

Counter({'국민': 6, '원내': 4, '개혁': 3, '구성': 3, '국회': 3, '대표': 3, '수석': 3, '개헌안': 2, '금주': 2, '노력': 2, '마련': 2, '우선': 2, '정치': 2, '제안': 2, '특위': 2, '활동': 2, '가능': 1, '강조': 1, '개선': 1, '개정': 1, '개헌': 1, '거듭': 1, '경제': 1, '고위': 1, '공수': 1, '공직자': 1, '과제': 1, '국민투표': 1, '기사': 1, '꼽았다': 1, '내야': 1, '다음': 1, '대책': 1, '도려': 1, '도약': 1, '도적': 1, '래야': 1, '마찬가지': 1, '문대': 1, '문의': 1, '민생': 1, '민주당': 1, '바란': 1, '박홍근': 1, '부응': 1, '부터': 1, '불어': 1, '비리': 1, '사법': 1, '새해': 1, '서울': 1, '설치': 1, '성과': 1, '수사': 1, '실시': 1, '아침': 1, '여야': 1, '완료': 1, '요구': 1, '원내대표': 1, '위도': 1, '의정': 1, '일자리': 1, '임시국회': 1, '입법': 1, '적절': 1, '적폐': 1, '접근': 1, '정책': 1, '정치권': 1, '제보': 1, '지방선거': 1, '차원': 1, '찬성': 1, '채팅': 1, '청산': 1, '촉구': 1, '최신': 1, '최우': 1, '트렌드': 1, '합의': 1, '헌법': 1, '혜림': 1, '확실': 1, '환부': 1, '활성화': 1, '회의': 1, '희망': 1})
국민
원내
개혁
구성
국회
대표
수석
개헌안
금주
노력
마련
우선
정치
제안
특위
활동
가능
강조
개선
개정
개헌
거듭
경제
고위
공수
공직자
26366
